# MSc Project - Questionnaire Pre Processing

In [1534]:
import pandas as pd
import sys
import numpy as np
from getmaps import field_mapping, occupation_mapping, education_mapping # These dictionaries contain answers from participants when they answered "other" and wrote their answer in

In [1535]:
# Check python version.
# This code should run for python version >=3.6

print("python", sys.version)

python 3.9.13 (main, Aug 25 2022, 18:29:29) 
[Clang 12.0.0 ]


In [1536]:
data = pd.read_csv("data/questionnaire.csv", sep=";", index_col=0)

In [1537]:
data_df = data.copy()

## Clean up

In [1538]:
data_df = data.copy()

# Removing rows where Start Time is before 3 PM on November 29 2024
data_df['Start time'] = pd.to_datetime(data_df['Start time'], format='%d.%m.%Y %H:%M')
data_df = data_df[data_df['Start time'] > pd.to_datetime('2024-11-29 15:00:00')]

# Removing unnecessary columns
data_df = data_df.drop([
    'Start time', 
    'Completion time', 
    'Email', 
    'Name',
    'I agree to participate in this study / Ég samþykki að taka þátt í þessari könnun'
    ], axis=1)

**Removing participants that did not answer the question "Hver er/vat þín námsbraut?" as it is neccessary to know what the participant studies for their answers to be relevant**

In [1539]:
data_df = data_df[~((data_df['Hver er/var þín námsbraut?'].isna()) & (data_df['In what language would you like to complete this questionnaire? / Á hvaða tungumáli vilt þú svara þessari könnun?'] == 'Íslensku'))]

**Changing the structure of the dataframe so that answers in english and icelandic are in the same columns**

In [1540]:
# Splitting the dataframe into two, one with Icelandic answers and the other with English answers
df_ice = data_df.loc[:,:'Er eitthvað sem þú vilt bæta við?']
df_en = data_df.loc[:,'Gender':] 

In [1541]:
lang_q = 'In what language would you like to complete this questionnaire? / Á hvaða tungumáli vilt þú svara þessari könnun?'
df_ice['Language'] = df_ice[lang_q]
df_ice = df_ice.drop([lang_q], axis=1)
df_ice = df_ice[df_ice['Language'] != 'English']

df_ice['Language'] = 'Icelandic'
df_en['Language'] = 'English'

df_en.drop(index=df_en.index[df_en.index.isin(df_ice.index)], inplace=True)

**Combine data for Icelandic and English participants**

In [1542]:
df_en.columns = df_ice.columns.tolist()

# Combining df_ice and df_en
df_combined = pd.concat([df_ice, df_en], ignore_index=True)

**Making 'id' a column**

In [1543]:
df_combined.index.name = 'id'
df_combined = df_combined.reset_index()

**Renaming and translating columns** 

In [1544]:
# Simplifying process of identifying questions relating to parents
df_combined.rename(columns={
    'Hvert er kyn þitt?': 'Gender',
    'Hvert er fæðingarár þitt?': 'Year of Birth',
    'Hæsta menntunarstig': 'Education Level',
    'Ert þú í námi?': 'Student',
    'Hverskonar námi ert þú í?': 'Current Degree',
    'Hver er/var þín námsbraut?': 'Education Field',
    'Starfssvið': 'Occupational Field',
    'Hversu mörg systkini átt þú?': 'Number of Siblings',
    'Hve marga einstaklinga myndir þú telja vera aðalforráðamenn þínir í uppeldi þínu?': 'Number of Parents',
    'Hver eru tengsl þín við þetta foreldri/forráðamann?': 'One parent - Connection',
    'Hvert er kyn þeirra?': 'One parent - Gender',
    'Hvert er þeirra hæsta menntunarstig?': 'One parent - Education Level',
    'Hver var námsbraut þeirra?': 'One parent - Education Field',
    'Hvert er starfssvið þeirra?': 'One parent - Occupational Field',
    'Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?': 'One parent - Influence',
    'Fyrsta foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?': 'Two parents - First - Connection',
    'Fyrsta foreldri - Hvert er kyn þeirra?': 'Two parents - First - Gender',
    'Fyrsta foreldri - Hvert er þeirra hæsta menntunarstig?': 'Two parents - First - Education Level',
    'Fyrsta foreldri - Hver var námsbraut þeirra?': 'Two parents - First - Education Field',
    'Fyrsta foreldri - Hvert er starfssvið þeirra?': 'Two parents - First - Occupational Field',
    'Fyrsta foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?': 'Two parents - First - Influence',
    'Annað foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?': 'Two parents - Second - Connection',
    'Annað foreldri - Hvert er kyn þeirra?': 'Two parents - Second - Gender',
    'Annað foreldri - Hvert er þeirra hæsta menntunarstig?': 'Two parents - Second - Education Level',
    'Annað foreldri - Hver var námsbraut þeirra?': 'Two parents - Second - Education Field',
    'Annað foreldri - Hvert er starfssvið þeirra?': 'Two parents - Second - Occupational Field',
    'Annað foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?': 'Two parents - Second - Influence',
    'Fyrsta foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?1': 'Three parents - First - Connection',
    'Fyrsta foreldri - Hvert er kyn þeirra?1': 'Three parents - First - Gender',
    'Fyrsta foreldri - Hvert er þeirra hæsta menntunarstig?1': 'Three parents - First - Education Level',
    'Fyrsta foreldri - Hver var námsbraut þeirra?2': 'Three parents - First - Education Field',
    'Fyrsta foreldri - Hvert er starfssvið þeirra?1': 'Three parents - First - Occupational Field',
    'Fyrsta foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?1': 'Three parents - First - Influence',
    'Annað foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?1': 'Three parents - Second - Connection',
    'Annað foreldri - Hvert er kyn þeirra?1': 'Three parents - Second - Gender',
    'Annað foreldri - Hvert er þeirra hæsta menntunarstig?1': 'Three parents - Second - Education Level',
    'Annað foreldri - Hver var námsbraut þeirra?1': 'Three parents - Second - Education Field',
    'Annað foreldri - Hvert er starfssvið þeirra?1': 'Three parents - Second - Occupational Field',
    'Annað foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?1': 'Three parents - Second - Influence',
    'Þriðja foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?': 'Three parents - Third - Connection',
    'Þriðja foreldri - Hvert er kyn þeirra?': 'Three parents - Third - Gender',
    'Þriðja foreldri - Hvert er þeirra hæsta menntunarstig?': 'Three parents - Third - Education Level',
    'Þriðja foreldri - Hver var námsbraut þeirra?': 'Three parents - Third - Education Field',
    'Þriðja foreldri - Hvert er starfssvið þeirra?': 'Three parents - Third - Occupational Field',
    'Þriðja foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?': 'Three parents - Third - Influence',
    'Fyrsta foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?2': 'Four parents - First - Connection',
    'Fyrsta foreldri - Hvert er kyn þeirra?2': 'Four parents - First - Gender',
    'Fyrsta foreldri - Hvert er þeirra hæsta menntunarstig?2': 'Four parents - First - Education Level',
    'Fyrsta foreldri - Hver var námsbraut þeirra?1': 'Four parents - First - Education Field',
    'Fyrsta foreldri - Hvert er starfssvið þeirra?2': 'Four parents - First - Occupational Field',
    'Fyrsta foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?2': 'Four parents - First - Influence',
    'Annað foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?2': 'Four parents - Second - Connection',
    'Annað foreldri - Hvert er kyn þeirra?2': 'Four parents - Second - Gender',
    'Annað foreldri - Hvert er þeirra hæsta menntunarstig?2': 'Four parents - Second - Education Level',
    'Annað foreldri - Hver var námsbraut þeirra?2': 'Four parents - Second - Education Field',
    'Annað foreldri - Hvert er starfssvið þeirra?2': 'Four parents - Second - Occupational Field',
    'Annað foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?2': 'Four parents - Second - Influence',
    'Þriðja foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?1': 'Four parents - Third - Connection',
    'Þriðja foreldri - Hvert er kyn þeirra?1': 'Four parents - Third - Gender',
    'Þriðja foreldri - Hvert er þeirra hæsta menntunarstig?1': 'Four parents - Third - Education Level',
    'Þriðja foreldri - Hver var námsbraut þeirra?1': 'Four parents - Third - Education Field',
    'Þriðja foreldri - Hvert er starfssvið þeirra?1': 'Four parents - Third - Occupational Field',
    'Þriðja foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?1': 'Four parents - Third - Influence',
    'Fjórða foreldri - Hver eru tengsl þín við þetta foreldri/forráðamann?': 'Four parents - Fourth - Connection',
    'Fjórða foreldri - Hvert er kyn þeirra?': 'Four parents - Fourth - Gender',
    'Fjórða foreldri - Hvert er þeirra hæsta menntunarstig?': 'Four parents - Fourth - Education Level',
    'Fjórða foreldri - Hver var námsbraut þeirra?': 'Four parents - Fourth - Education Field',
    'Fjórða foreldri - Hvert er starfssvið þeirra?': 'Four parents - Fourth - Occupational Field',
    'Fjórða foreldri - Telur þú að þessi foreldri/forráðamaður hafi haft áhrif á þig við val á námi þínu að einhverju leiti?': 'Four parents - Fourth - Influence',
    'Er einhver í fjölskyldu þinni, svo sem systkini, afi, amma, frændi, frænka eða frændi, sem hefur stundað sama nám og þú?': 'Same Education as Family Member',
    'Hvaða fjölskyldu meðlimur hefur stundað sama nám og þú? Nefndu alla sem eiga við': 'Family Members w Same Education',
    'Er einhver í fjölskyldu þinni, svo sem systkini, afi, amma, frænka eða frændi, sem hefur unnið í sama fagi eða innan sama starfsviðs og þú?': 'Same Occupation as Family Member',
    'Hvaða fjölskyldu meðlimur hefur unnið í sama fagi eða innan sama starfsviðs og þú? Nefndu alla sem eiga við': 'Family Members w Same Occupation',
    'Telur þú að það fag sem þú valdir sé á einhvern hátt "hefð" innan fjölskyldu þinnar?': 'Tradition',
    'Er eitthvað sem þú vilt bæta við?': 'Comments'
}, inplace=True)

## Transform dataframe for better clarity

In [1545]:
# Function to transform the dataframe
def reshape_df(df):

    single_parent_cols = [
        "One parent - Connection", "One parent - Gender", "One parent - Education Level",
        "One parent - Education Field", "One parent - Occupational Field", "One parent - Influence"
    ]
    
    two_parent_cols = [
        ["Two parents - First - Connection", "Two parents - First - Gender", "Two parents - First - Education Level",
         "Two parents - First - Education Field", "Two parents - First - Occupational Field", "Two parents - First - Influence"],
        ["Two parents - Second - Connection", "Two parents - Second - Gender", "Two parents - Second - Education Level",
         "Two parents - Second - Education Field", "Two parents - Second - Occupational Field", "Two parents - Second - Influence"]
    ]

    three_parent_cols = [
        ["Three parents - First - Connection", "Three parents - First - Gender", "Three parents - First - Education Level",
         "Three parents - First - Education Field", "Three parents - First - Occupational Field", "Three parents - First - Influence"],
        ["Three parents - Second - Connection", "Three parents - Second - Gender", "Three parents - Second - Education Level",
         "Three parents - Second - Education Field", "Three parents - Second - Occupational Field", "Three parents - Second - Influence"],
        ["Three parents - Third - Connection", "Three parents - Third - Gender", "Three parents - Third - Education Level",
         "Three parents - Third - Education Field", "Three parents - Third - Occupational Field", "Three parents - Third - Influence"]
    ]

    four_parent_cols = [
        ["Four parents - First - Connection", "Four parents - First - Gender", "Four parents - First - Education Level",
         "Four parents - First - Education Field", "Four parents - First - Occupational Field", "Four parents - First - Influence"],
        ["Four parents - Second - Connection", "Four parents - Second - Gender", "Four parents - Second - Education Level",
         "Four parents - Second - Education Field", "Four parents - Second - Occupational Field", "Four parents - Second - Influence"],
        ["Four parents - Third - Connection", "Four parents - Third - Gender", "Four parents - Third - Education Level",
         "Four parents - Third - Education Field", "Four parents - Third - Occupational Field", "Four parents - Third - Influence"],
        ["Four parents - Fourth - Connection", "Four parents - Fourth - Gender", "Four parents - Fourth - Education Level",
         "Four parents - Fourth - Education Field", "Four parents - Fourth - Occupational Field", "Four parents - Fourth - Influence"]
    ]
    # List to hold transformed rows
    transformed_rows = []

    for _, row in df.iterrows():
        base_info = row[["id", "Gender", "Year of Birth", "Education Level", "Student", "Current Degree", 
                         "Education Field", "Occupational Field", "Number of Siblings", "Number of Parents",
                         "Same Education as Family Member", "Family Members w Same Education", "Same Occupation as Family Member",
                         "Family Members w Same Occupation", "Tradition", "Comments", "Language",
                         ]].to_dict()
        
        if row["Number of Parents"] == 1:
            parent_info = row[single_parent_cols].to_dict()
            parent_info = {k.replace("One parent - ", "Parent "): v for k, v in parent_info.items()}
            transformed_rows.append({**base_info, **parent_info})
        
        elif row["Number of Parents"] == 2:
            for parent_set in two_parent_cols:
                parent_info = row[parent_set].to_dict()
                parent_info = {k.replace("Two parents - First - ", "Parent ").replace("Two parents - Second - ", "Parent "): v for k, v in parent_info.items()}
                transformed_rows.append({**base_info, **parent_info})

        elif row["Number of Parents"] == 3:
            for parent_set in three_parent_cols:
                parent_info = row[parent_set].to_dict()
                parent_info = {k.replace("Three parents - First - ", "Parent ").replace("Three parents - Second - ", "Parent ").replace("Three parents - Third - ", "Parent "): v for k, v in parent_info.items()}
                transformed_rows.append({**base_info, **parent_info})

        elif row["Number of Parents"] == 4:
            for parent_set in four_parent_cols:
                parent_info = row[parent_set].to_dict()
                parent_info = {k.replace("Four parents - First - ", "Parent ").replace("Four parents - Second - ", "Parent ").replace("Four parents - Third - ", "Parent ").replace("Four parents - Fourth - ", "Parent "): v for k, v in parent_info.items()}
                transformed_rows.append({**base_info, **parent_info})
    
    return pd.DataFrame(transformed_rows)


In [1546]:
# Apply transformation
df_transformed = reshape_df(df_combined)

# Rename columns for consistency
df_transformed.rename(columns={
    "Parent Connection": "Connection",
    "Parent Gender": "Parent Gender",
    "Parent Education Level": "Parent Education Level",
    "Parent Education Field": "Parent Education Field",
    "Parent Occupational Field": "Parent Occupational Field",
    "Parent Influence": "Influence"
}, inplace=True)

**Column cleanup - Gender**

In [1547]:
gender_mapping = {"Karl": "Man", "Kona": "Woman", np.nan: "Unknown", "Kvár": "Non-Binary"}

df_transformed["Gender"] = df_transformed["Gender"].replace(gender_mapping)
df_transformed["Parent Gender"] = df_transformed["Parent Gender"].replace(gender_mapping)

**Column cleanup - Year of Birth**

Some participants answered in the form of YYYY e.g. 1990 while others answered in the form of YY e.g. 90. The following function makes sure the year is in a standard format.

In [1548]:
def convert_year(yob):
    try:
        yob = int(yob)  # Ensure it's an integer
        if yob < 100:  # If it's a two-digit year
            if yob >= 30:  # Assume 1930-1999
                yob += 1900
            else:  # Assume 2000-2029
                yob += 2000
        return yob
    except (ValueError, TypeError):  # Handle NaN or non-numeric values
        return None  # Keep as None (NaN in pandas)


df_transformed["Year of Birth"] = df_transformed["Year of Birth"].apply(convert_year)

Adding "Age" column

In [1549]:
# Calculate Age as of 2025
df_transformed["Age"] = 2025 - df_transformed["Year of Birth"]

**Column cleanup - Education Level**

In [1550]:
education_map = {
    'Grunnmenntun (t.d., Grunnskólapróf, gagnfræðipróf)': 'Primary',
    'Starfs- og framhaldsmenntun (t.d., Stúdentspróf, sveinspróf)': 'Secondary',
    'Grunnnám á háskólastigi\xa0(t.d., BA, BS)': 'Bachelors',
    'Meistaranám á háskólastigi\xa0(t.d., MA, MS, MBA)': 'Masters',
    'Doktorsnám\xa0(t.d., PhD, EdD)': 'Doctoral',
    'High school diploma, or vocational degree at the upper secondary level (e.g., Stúdentspróf, Sveinspróf)': 'Secondary',
    'Bachelor’s degree (e.g., BA, BS)': 'Bachelors',
    'Master’s degree (e.g., MA, MS, MBA)': 'Masters',
    'Doctoral degree (e.g., PhD, EdD)': 'Doctoral',
}


df_transformed['Education Level'] = df_transformed['Education Level'].map(education_map)
df_transformed['Parent Education Level'] = df_transformed['Parent Education Level'].map(education_map)

**Column cleanup - Student**

In [1551]:
yes_no_mapping = {"Nei": "No", "Já": "Yes", np.nan: "Unknown", "Kannski": "Maybe"}

df_transformed["Student"] = df_transformed["Student"].replace(yes_no_mapping)

**Column cleanup - Current Degree**

In [ ]:
def map_education(value):
    return education_mapping.get(value, "Other")

df_transformed['Current Degree'] = df_transformed['Current Degree'].apply(map_education)

**Column cleanup - Education Field & Parent Education Field**

In [1553]:
education_field_list = [
    "Afbrotafræði", "Alþjóðaréttur", "Alþjóðasamskipti", "Alþjóðaviðskipti", "Arkitektúr",
    "Borgarskipulag", "Byggingarverkfræði", "Bókhald", "Bókmenntafræði", "Dýralæknisfræði",
    "Efnafræði", "Eðlisfræði", "Ferðamálafræði", "Fjármál", "Fjármálaverkfræði", "Flugverkfræði",
    "Fornleifafræði", "Framhaldsskóla kennsla", "Frammistöðulistir (leikhús, dans o.s.frv.)",
    "Frumkvöðlafræði", "Fyrirtækjafjármál", "Félagsfræði", "Félagsráðgjöf", "Geislafræði", 
    "Grafísk hönnun", "Grunnskóla kennsla", "Guðfræði & trúarbragðafræði", "Hagfræði", "Heilbrigðismál",
    "Heimspeki", "Hjúkrunarfræði", "Hreyfifræði", "Hugbúnaðarverkfræði", "Iðnaðarlíftækni"
    "Jarðvísindi (jarðfræði, veðurfræði, ofl.)", "Kennsla", "Kvikmyndafræði", "Landafræði",
    "Landbúnaðarfræði", "Landslagsarkitektúr", "Leikskóla kennsla", "List og hönnun", "Lyfjafræði", 
    "Læknisfræði (MD)", "Lífefnafræði", "Lífeindafræði", "Líffræði", "Lögfræði", "Mannauðsstjórnun", 
    "Mannfræði", "Markaðsfræði", "Mekatróníkverkfræði", "Menntastjórnun", "Miðlun og samskipti", 
    "Næringarfræði og næringarráðgjöf", "Opinber stjórnsýsla", "Rafmagnsverkfræði", "Refsiréttur", "Saga",
    "Sameindalíffræði", "Sjúkraþjálfun", "Skógarbúskapur", "Stjórnmálafræði", "Stjörnufræði", "Stærðfræði", 
    "Sálfræði", "Sérkennsla", "Tannlækningar", "Trúarbragðafræði", "Tungumál og bókmenntir", "Tónlist",
    "Tölvunarfræði", "Umhverfisfræði", "Umhverfisstjórnun", "Upplýsingatækni", "Verkfræði", "Verkfræðistjórnun",
    "Viðburðastjórnun", "Viðskiptafræði", "Viðskiptastjórnun", "Vélaverkfræði", "Vélmennafræði (Robotics)",
    "Íþróttafræði", "Íþróttastjórnun", "Íþróttaþjálfun", "Þjóðfræði", "Other"
    ]


In [ ]:
# Replacing values not in education_field_list with their appropriate value

def map_field(value):
    # Check if the value is in field_mapping or in education_field_list
    if value in field_mapping:
        return field_mapping[value]
    elif value in education_field_list:
        return value 
    else:
        return "Other" 

df_transformed['Education Field'] = df_transformed['Education Field'].apply(map_field)
df_transformed['Parent Education Field'] = df_transformed['Parent Education Field'].apply(map_field)


**Column cleanup - Occupational Field & Parent Occupational Field**

In [1555]:
occupation_list = [
    "Student", "Agriculture, forestry, and fisheries", "Arts, and entertainment", "Business and management", "Construction and real estate"
    "Education and training", "Engineering and manufacturing", "Finance and banking", "Government, public administration, and law enforcement",
    "Healthcare and medicine", "Hospitality and tourism", "Information technology and software", "Law and legal services",
    "Nonprofit and social services", "Retail and consumer services", "Sales, marketing, and advertising", "Science and research", "Telecommunications and media"
    ]

In [1556]:
occupation_map = {
    'Ég er námsmaður': 'Student',
    'Landbúnaður, skógrækt og fiskveiðar': 'Agriculture, forestry, and fisheries',
    'Listir og afþreying': 'Arts, and entertainmen',
    'Viðskipti og stjórnun': 'Business and management',
    'Byggingariðnaður og fasteignir': 'Construction and real estate',
    'Menntun og þjálfun': 'Education and training',
    'Verkfræði og framleiðsla': 'Engineering and manufacturing',
    'Fjármögnun og bankastarfsemi': 'Finance and banking',
    'Ríkisvöld, opinber stjórnsýsla og löggæsla': 'Government, public administration, and law enforcement',
    'Heilbrigðisþjónusta og læknisfræði': 'Healthcare and medicine',
    'Gestamóttaka og ferðaþjónusta': 'Hospitality and tourism',
    'Upplýsingatækni og hugbúnaður': 'Information technology and software',
    'Lögfræði og lögfræðiþjónustur': 'Law and legal services',
    'Óhagnaðardrifin samtök og félagsþjónusta': 'Nonprofit and social services',
    'Smásala og neytendþjónusta': 'Retail and consumer services',
    'Sölur, markaðssetning og auglýsingar': 'Sales, marketing, and advertising',
    'Vísindi og rannsóknir': 'Science and research',
    'Fjarskipti og fjölmiðlar': 'Telecommunications and media',
}

df_transformed["Occupational Field"] = df_transformed["Occupational Field"].replace(occupation_map)

In [1557]:
def map_occupation(value):
    return occupation_mapping.get(value, "Other")

df_transformed['Occupational Field'] = df_transformed['Occupational Field'].apply(map_occupation)
df_transformed['Parent Occupational Field'] = df_transformed['Parent Occupational Field'].apply(map_occupation)

**Column cleanup - Same Education as Family Member**

In [1558]:
df_transformed["Same Education as Family Member"] = df_transformed["Same Education as Family Member"].replace(yes_no_mapping)


**Column cleanup - Same Occupation as Family Member**

In [1559]:
df_transformed["Same Occupation as Family Member"] = df_transformed["Same Occupation as Family Member"].replace(yes_no_mapping)

**Column cleanup - Tradition**

In [1560]:
df_transformed["Tradition"] = df_transformed["Tradition"].replace(yes_no_mapping)

**Column cleanup - Connection**

In [1561]:
connection_mapping = {
    "Móðir": "Mother", "Faðir": "Father", "Frænka": "Aunt", "Amma": "Grandma", 
    "Stjúp-faðir": "Step-Father", "Stjúp-móðir": "Step-Mother", "Afi": "Grandpa", 
    "Systir": "Sister", "Fósturmóðir": "Foster-Mother", "Fósturfaðir": "Foster-Father", np.nan: "Unknown"
}
df_transformed["Connection"] = df_transformed["Connection"].replace(connection_mapping)

**Column cleanup - Influence**

In [1562]:
df_transformed["Influence"] = df_transformed["Influence"].replace(yes_no_mapping)

## Defining What Degrees belong to the same educationnal group of closeness

**Health**
- Íþróttaþjálfun - Athletic Training
- Tannlækningar - Dentistry
- Læknisfræði (MD) - Medicine (MD)
- Hjúkrunarfræði - Nursing
- Næringarfræði og næringarráðgjöf - Nutrition and Dietetics
- Lyfjafræði - Pharmacy
- Sjúkraþjálfun - Physical Therapy
- Geislafræði - Radiology
- Íþróttafræði - Sport Science
- Dýralæknisfræði - Veterinary Medicine
- Heilbrigðismál - Public Health
- Hreyfifræði - Kinesiology


**Education**
- Leikskóla kennsla - Early Childhood Education
- Kennsla - Education
- Grunnskóla kennsla - Elementary Education
- Framhaldsskóla kennsla - Secondary Education 
- Sérkennsla - Special Education
- Menntastjórnun - Education Management

**Social sciences**
- Mannfræði - Anthropology
- Fornleifafræði - Archaeology
- Afbrotafræði - Criminology
- Refsiréttur - Criminal Justice
- Félagsfræði - Sociology
- Stjórnmálafræði - Political Science
- Public Administration
- Félagsráðgjöf - Social Work
- Þjóðfræði - Ethnology
- Sálfræði - Psychology
- Opinber stjórnsýsla - Public Administration
- Alþjóðasamskipti - International Relations
- Borgarskipulag - Urban Planning
- Ferðamálafræði - Tourism Studies
- Landafræði - Geography

**Arts and humanities**
- Grafísk hönnun - Graphic Design
- Frammistöðulistir (leikhús, dans o.s.frv.) - Performing Arts (Theater, Dance, etc.)
- Guðfræði & trúarbragðafræði - Theology & Religious Studies
- Trúarbragðafræði - Religious Studies
- Tónlist - Music
- Kvikmyndafræði - Film Studies
- Heimspeki - Philosophy
- Saga - History
- Tungumál og bókmenntir - Languages and Literature
- Bókmenntafræði - Library Studies
- List og hönnun - Art and Design
- Miðlun og samskipti - Media and communication


**Business and Economics**
- Bókhald - Accounting
- Viðskiptafræði - Business
- Viðskiptastjórnun - Business Administration
- Fyrirtækjafjármál - Corporate Finance
- Hagfræði - Economics
- Viðburðastjórnun - Event Management
- Fjármál - Finance
- Alþjóðaviðskipti - International Business
- Markaðsfræði - Marketing
- Mannauðsstjórnun - Human Resource Management
- Frumkvöðlafræði - Entrepreneurship
- Alþjóðaréttur - International Law
- Íþróttastjórnun - Sports Management
- Lögfræði - Law


**Natural sciences**
- Lífefnafræði - Biochemistry
- Líffræði - Biology
- Efnafræði - Chemistry
- Jarðvísindi (jarðfræði, veðurfræði, ofl.) - Earth Science (Geology, Meteorology, etc.)
- Eðlisfræði - Physics
- Lífeindafræði - Microbiology
- Sameindalíffræði - Molecular Biology 
- Stjörnufræði - Astronomy
- Stærðfræði - Mathematics
- Umhverfisfræði - Environmental Science
- Skógarbúskapur - Forestry
- Landbúnaðarfræði - Agriculture

**Engineering**
- Flugverkfræði - Aerospace Engineering
- Byggingarverkfræði - Civil Engineering
- Rafmagnsverkfræði - Electrical Engineering
- Verkfræði - Engineering
- Verkfræðistjórnun - Engineering Management
- Vélaverkfræði - Mechanical Engineering
- Mekatróníkverkfræði - Mechatronics Engineerin
- Iðnfræði - Industrial Engineering
- Fjármálaverkfræði - Financial Engineering
- Arkitektúr - Architecture
- Tæknifræði - Applied Engineering

**Technology** 
- Vélmennafræði (Robotics) - Robotics
- Iðnaðarlíftækni - Industrial Biotechnology
- Upplýsingatækni - Information Technology
- Tölvunarfræði - Computer Science
- Hugbúnaðarverkfræði - Software Engineering




In [1563]:
# Define the field-to-group mapping
field_to_group = {
    # Health group
    "Íþróttaþjálfun": "Health",
    "Tannlækningar": "Health",
    "Læknisfræði (MD)": "Health",
    "Hjúkrunarfræði": "Health",
    "Næringarfræði og næringarráðgjöf": "Health",
    "Lyfjafræði": "Health",
    "Sjúkraþjálfun": "Health",
    "Geislafræði": "Health",
    "Íþróttafræði": "Health",
    "Dýralæknisfræði": "Health",
    "Heilbrigðismál": "Health",
    "Hreyfifræði": "Health",
    
    # Education group
    "Leikskóla kennsla": "Education",
    "Kennsla": "Education",
    "Grunnskóla kennsla": "Education",
    "Framhaldsskóla kennsla": "Education",
    "Sérkennsla": "Education",
    "Menntastjórnun": "Education",
    
    # Social science group
    "Mannfræði": "Social sciences",
    "Fornleifafræði": "Social sciences",
    "Afbrotafræði": "Social sciences",
    "Refsiréttur": "Social sciences",
    "Félagsfræði": "Social sciences",
    "Stjórnmálafræði": "Social sciences",

    "Félagsráðgjöf": "Social sciences",
    "Þjóðfræði": "Social sciences",
    "Sálfræði": "Social sciences",
    "Opinber stjórnsýsla": "Social sciences",
    "Alþjóðasamskipti": "Social sciences",
    "Borgarskipulag": "Social sciences",
    "Ferðamálafræði": "Social sciences",
    "Landafræði": "Social sciences",
    
    # Arts and humanities group
    "Grafísk hönnun": "Arts and humanities",
    "Frammistöðulistir (leikhús, dans o.s.frv.)": "Arts and humanities",
    "Guðfræði & trúarbragðafræði": "Arts and humanities",
    "Trúarbragðafræði": "Arts and humanities",
    "Tónlist": "Arts and humanities",
    "Kvikmyndafræði": "Arts and humanities",
    "Heimspeki": "Arts and humanities",
    "Saga": "Arts and humanities",
    "Tungumál og bókmenntir": "Arts and humanities",
    "Bókmenntafræði": "Arts and humanities",
    "List og hönnun": "Arts and humanities",
    "Miðlun og samskipti": "Arts and humanities",
    
    # Business and Economics group
    "Bókhald": "Business and economics",
    "Viðskiptafræði": "Business and economics",
    "Viðskiptastjórnun": "Business and economics",
    "Fyrirtækjafjármál": "Business and economics",
    "Hagfræði": "Business and economics",
    "Viðburðastjórnun": "Business and economics",
    "Fjármál": "Business and economics",
    "Alþjóðaviðskipti": "Business and economics",
    "Markaðsfræði": "Business and economics",
    "Mannauðsstjórnun": "Business and economics",
    "Frumkvöðlafræði": "Business and economics",
    "Alþjóðaréttur": "Business and economics",
    "Íþróttastjórnun": "Business and economics",
    "Lögfræði": "Business and economics",
    
    # Natural sciences group
    "Lífefnafræði": "Natural sciences",
    "Líffræði": "Natural sciences",
    "Efnafræði": "Natural sciences",
    "Jarðvísindi (jarðfræði, veðurfræði, ofl.)": "Natural sciences",
    "Eðlisfræði": "Natural sciences",
    "Lífeindafræði": "Natural sciences",
    "Sameindalíffræði": "Natural sciences",
    "Stjörnufræði": "Natural sciences",
    "Stærðfræði": "Natural sciences",
    "Umhverfisfræði": "Natural sciences",
    "Skógarbúskapur": "Natural sciences",
    "Landbúnaðarfræði": "Natural sciences",

    # Engineering group
    "Flugverkfræði": "Engineering",
    "Byggingarverkfræði": "Engineering",
    "Rafmagnsverkfræði": "Engineering",
    "Verkfræði": "Engineering",
    "Verkfræðistjórnun": "Engineering",
    "Vélaverkfræði": "Engineering",
    "Mekatróníkverkfræði": "Engineering",
    "Iðnaðarlíftækni": "Engineering",
    "Fjármálaverkfræði": "Engineering",
    "Arkitektúr": "Engineering",
    "Tæknifræði": "Engineering",

    # Technology group
    "Upplýsingatækni": "Technology",
    "Hugbúnaðarverkfræði": "Technology",
    "Vélmennafræði (Robotics)": "Technology",
    "Iðnfræði": "Technology",
    "Tölvunarfræði": "Technology",
    
    
    # Other group
    "Other": "Other"
}

# Map the fields to the corresponding group
df_transformed["Education Group"] = df_transformed["Education Field"].map(field_to_group)
df_transformed["Parent Education Group"] = df_transformed["Parent Education Field"].map(field_to_group)


### Handling NaN values

In [1564]:
print(df_transformed.isnull().sum())

id                                    0
Gender                                0
Year of Birth                         8
Education Level                       0
Student                               0
Current Degree                        0
Education Field                       0
Occupational Field                    0
Number of Siblings                   20
Number of Parents                     0
Same Education as Family Member       0
Family Members w Same Education     502
Same Occupation as Family Member      0
Family Members w Same Occupation    475
Tradition                             0
Comments                            558
Language                              0
Connection                            0
Parent Gender                         0
Parent Education Level                3
Parent Education Field                0
Parent Occupational Field             0
Influence                             0
Age                                   8
Education Group                       0


In [1565]:
df_transformed["Number of Siblings"].fillna(df_transformed["Number of Siblings"].mean(), inplace=True) 
df_transformed["Family Members w Same Education"].fillna('', inplace=True) 
df_transformed["Family Members w Same Occupation"].fillna('', inplace=True) 
df_transformed["Comments"].fillna('', inplace=True) 
df_transformed["Parent Education Level"].fillna('Unknown', inplace=True)
df_transformed["Age"].fillna(df_transformed["Age"].mean(), inplace=True) 

## Adding Label

In [1568]:
labelling = df_transformed.copy()
labelling.columns = labelling.columns.str.replace(' ', '')

In [1569]:
labelling = labelling[(labelling['Gender'] == 'Woman') | (labelling['Gender'] == 'Man')]

labelling['FemaleParentEducationLevel'] = labelling.apply(lambda row: row['ParentEducationLevel'] if row['ParentGender'] == 'Woman' else None, axis=1)
labelling['MaleParentEducationLevel'] = labelling.apply(lambda row: row['ParentEducationLevel'] if row['ParentGender'] == 'Man' else None, axis=1)
labelling['FemaleParentEducationGroup'] = labelling.apply(lambda row: row['ParentEducationGroup'] if row['ParentGender'] == 'Woman' else None, axis=1)
labelling['MaleParentEducationGroup'] = labelling.apply(lambda row: row['ParentEducationGroup'] if row['ParentGender'] == 'Man' else None, axis=1)


# TESTING
labelling['FemaleParentInfluence'] = labelling.apply(lambda row: row['Influence'] if row['ParentGender'] == 'Woman' else None, axis=1)
labelling['MaleParentInfluence'] = labelling.apply(lambda row: row['Influence'] if row['ParentGender'] == 'Man' else None, axis=1)

In [1570]:
labelling['NumberofParents'].value_counts()

2.0    624
3.0     60
4.0     36
1.0     23
Name: NumberofParents, dtype: int64

**Logic when selecting the male and female parents that represent a person**

- If person has 1 female parent -> That parents education becomes FemaleParentEducationGroup and FemaleParentEducationLevel
- If person has 1 male parent -> That parents education becomes MaleParentEducationGroup and MaleParentEducationLevel

- If person has more than 1 female parent 

    - Check if any of those parents has the same education group as the participant

        - If yes -> that parents education group becomes FemaleParentEducationGroup

        - If no -> choose parent with higher education level to represent FemaleParentEducationGroup, if they have the same education level -> choose the first one

- If person has more than 1 male parent 

    - Check if any of those parents has the same education group as the participant#       

        - If yes -> that parents education group becomes MaleParentEducationGroup

        - If no -> choose parent with higher education level to represent MaleParentEducationGroup, if they have the same education level -> choose the first one


In [1571]:

def select_parent_education(df, group_col, level_col, influ_col, participant_edu_col):
    if len(df) == 1:
        return df.iloc[0][group_col], df.iloc[0][level_col], df.iloc[0][influ_col]
    
    participant_edu = df[participant_edu_col].iloc[0]
    same_group_match = df[df[group_col] == participant_edu]
    
    if not same_group_match.empty:
        return same_group_match.iloc[0][group_col], same_group_match.iloc[0][level_col], same_group_match.iloc[0][influ_col]
    
    return df.sort_values(by=level_col, ascending=False).iloc[0][[group_col, level_col, influ_col]].values

def process_parent_education(df):
    result = []
    cols = {
        "FemaleParentEducationGroup", "FemaleParentEducationLevel", "MaleParentEducationGroup", "MaleParentEducationLevel", 
        "FemaleParentInfluence", "MaleParentInfluence",
        }
    extra_columns = [col for col in df.columns if col not in cols]
    
    for person_id, group in df.groupby("id"):
        female_parents = group.dropna(subset=["FemaleParentEducationGroup"])
        male_parents = group.dropna(subset=["MaleParentEducationGroup"])
        
        female_group, female_level, female_influ = (None, None, None)
        male_group, male_level, male_influ = (None, None, None)
        
        if not female_parents.empty:
            female_group, female_level, female_influ = select_parent_education(
                female_parents, "FemaleParentEducationGroup", "FemaleParentEducationLevel", "FemaleParentInfluence", "EducationGroup"
            )
        
        if not male_parents.empty:
            male_group, male_level, male_influ = select_parent_education(
                male_parents, "MaleParentEducationGroup", "MaleParentEducationLevel", "MaleParentInfluence", "EducationGroup"
            )
        
        extra_data = group.iloc[0][extra_columns].to_dict()
        
        result.append({
            "id": person_id,
            **extra_data,
            "FemaleParentEducationGroup": female_group,
            "FemaleParentEducationLevel": female_level,
            "MaleParentEducationGroup": male_group,
            "MaleParentEducationLevel": male_level,
            "FemaleParentInfluence": female_influ,
            "MaleParentInfluence": male_influ,
        })
    
    return pd.DataFrame(result)

In [1572]:
labelling = labelling[[
    'id', 'Gender', 'Age', 'EducationLevel', 'EducationGroup', 
    'Student',
    'NumberofSiblings', 'NumberofParents',
    'FemaleParentEducationLevel','MaleParentEducationLevel', 
    'FemaleParentEducationGroup','MaleParentEducationGroup',
    'FemaleParentInfluence', 'MaleParentInfluence'
    ]].copy()

In [1573]:
processed_labelling = process_parent_education(labelling)

In [1574]:
def assign_label(row):
    if (row["EducationGroup"] == row["FemaleParentEducationGroup"]) & (row["EducationGroup"] != row["MaleParentEducationGroup"]):
        return 1
    elif (row["EducationGroup"] == row["MaleParentEducationGroup"]) & (row["EducationGroup"] != row["FemaleParentEducationGroup"]):
        return 2
    elif (row["EducationGroup"] == row["FemaleParentEducationGroup"]) & (row["EducationGroup"] == row["MaleParentEducationGroup"]):
        return 3
    else:
        return 0 

processed_labelling["Label"] = processed_labelling.apply(assign_label, axis=1)

**Removing those with the eudcation groups Education and Natural sciences**

In [1575]:
processed_labelling = processed_labelling[(processed_labelling['EducationGroup']!= 'Education') & (processed_labelling['EducationGroup']!= 'Natural sciences')].copy()

In [ ]:
processed_labelling.to_pickle("processed_data.pkl")